<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [1]:
%%capture
!pip install --upgrade tensorflow

In [2]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

2024-11-19 07:34:21.706639: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 07:34:22.213478: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-19 07:34:22.228620: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-19 07:34:22.228668: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

2024-11-19 07:34:38.350078: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-11-19 07:34:38.350139: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-11-19 07:34:38.350166: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyterlab-mylokkies): /proc/driver/nvidia/version does not exist
2024-11-19 07:34:38.350766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [9]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [10]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 57s 69ms/step - loss: 0.6827 - accuracy: 0.5242 - val_loss: 0.6594 - val_accuracy: 0.6321
Epoch 2/10
782/782 [==============================] - 48s 61ms/step - loss: 0.6344 - accuracy: 0.6678 - val_loss: 0.6224 - val_accuracy: 0.7072
Epoch 3/10
782/782 [==============================] - 46s 59ms/step - loss: 0.6022 - accuracy: 0.7310 - val_loss: 0.6023 - val_accuracy: 0.7321
Epoch 4/10
782/782 [==============================] - 47s 60ms/step - loss: 0.5795 - accuracy: 0.7568 - val_loss: 0.5900 - val_accuracy: 0.7326
Epoch 5/10
782/782 [==============================] - 46s 59ms/step - loss: 0.5609 - accuracy: 0.7688 - val_loss: 0.5739 - val_accuracy: 0.7563
Epoch 6/10
782/782 [==============================] - 46s 59ms/step - loss: 0.5444 - accuracy: 0.7808 - val_loss: 0.5624 - val_accuracy: 0.7566
Epoch 7/10
782/782 [==============================] - 38s 49ms/step - loss: 0.5298 - accuracy: 0.7902 - val_loss: 0.5544 - val_accuracy:

In [11]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 6s 8ms/step - loss: 0.5267 - accuracy: 0.7689
Test score: 0.5266501307487488
Test accuracy: 0.7689200043678284


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [12]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [13]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 73s 92ms/step - loss: 0.6413 - accuracy: 0.6264 - val_loss: 0.5559 - val_accuracy: 0.7222
Epoch 2/10
782/782 [==============================] - 69s 88ms/step - loss: 0.4949 - accuracy: 0.7614 - val_loss: 0.4879 - val_accuracy: 0.7625
Epoch 3/10
782/782 [==============================] - 69s 88ms/step - loss: 0.4157 - accuracy: 0.8126 - val_loss: 0.4345 - val_accuracy: 0.7987
Epoch 4/10
782/782 [==============================] - 69s 88ms/step - loss: 0.3675 - accuracy: 0.8400 - val_loss: 0.4105 - val_accuracy: 0.8126
Epoch 5/10
782/782 [==============================] - 69s 88ms/step - loss: 0.3362 - accuracy: 0.8592 - val_loss: 0.3986 - val_accuracy: 0.8198
Epoch 6/10
782/782 [==============================] - 71s 91ms/step - loss: 0.3143 - accuracy: 0.8693 - val_loss: 0.3940 - val_accuracy: 0.8221
Epoch 7/10
782/782 [==============================] - 69s 89ms/step - loss: 0.2974 - accuracy: 0.8769 - val_loss: 0.3840 - val_accuracy:

In [16]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [17]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [18]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [19]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 96s 120ms/step - loss: 0.6743 - accuracy: 0.5628 - val_loss: 0.6193 - val_accuracy: 0.6566
Epoch 2/10
782/782 [==============================] - 91s 117ms/step - loss: 0.5507 - accuracy: 0.7218 - val_loss: 0.5102 - val_accuracy: 0.7482
Epoch 3/10
782/782 [==============================] - 58s 74ms/step - loss: 0.4620 - accuracy: 0.7849 - val_loss: 0.4722 - val_accuracy: 0.7740
Epoch 4/10
782/782 [==============================] - 57s 74ms/step - loss: 0.4149 - accuracy: 0.8139 - val_loss: 0.4408 - val_accuracy: 0.7957
Epoch 5/10
782/782 [==============================] - 58s 74ms/step - loss: 0.3843 - accuracy: 0.8308 - val_loss: 0.4547 - val_accuracy: 0.7886
Epoch 6/10
782/782 [==============================] - 58s 74ms/step - loss: 0.3652 - accuracy: 0.8408 - val_loss: 0.4002 - val_accuracy: 0.8204
Epoch 7/10
782/782 [==============================] - 58s 74ms/step - loss: 0.3505 - accuracy: 0.8481 - val_loss: 0.4056 - val_accurac

In [20]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 161s 206ms/step - loss: 0.3152 - accuracy: 0.8686 - val_loss: 0.3781 - val_accuracy: 0.8330
Epoch 2/10
782/782 [==============================] - 130s 167ms/step - loss: 0.3097 - accuracy: 0.8716 - val_loss: 0.3734 - val_accuracy: 0.8345
Epoch 3/10
782/782 [==============================] - 108s 138ms/step - loss: 0.3042 - accuracy: 0.8742 - val_loss: 0.3725 - val_accuracy: 0.8350
Epoch 4/10
782/782 [==============================] - 100s 128ms/step - loss: 0.2989 - accuracy: 0.8762 - val_loss: 0.3767 - val_accuracy: 0.8337
Epoch 5/10
782/782 [==============================] - 99s 126ms/step - loss: 0.2948 - accuracy: 0.8768 - val_loss: 0.3650 - val_accuracy: 0.8388
Epoch 6/10
782/782 [==============================] - 104s 133ms/step - loss: 0.2909 - accuracy: 0.8792 - val_loss: 0.3680 - val_accuracy: 0.8381
Epoch 7/10
782/782 [==============================] - 97s 124ms/step - loss: 0.2874 - accuracy: 0.8819 - val_loss: 0.3717 - v

---
### Machine Learning Foundation (C) 2020 IBM Corporation
